google colab notebook

# A simple classifier neural network in pytorch

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim

import pandas as pd
import numpy as np


In [ ]:
dataset = pd.read_csv('https://raw.githubusercontent.com/futurexskill/ml-model-deployment/main/storepurchasedata_large.csv')


In [ ]:
dataset.describe()

,Age,Salary,Purchased
count,1554.000000,1554.000000,1554.000000
mean,44.296010,57042.471042,0.694981
std,17.462458,21209.244800,0.460564
min,18.000000,20000.000000,0.000000
25%,27.000000,46000.000000,0.000000
50%,43.000000,60000.000000,1.000000
75%,62.000000,66000.000000,1.000000
max,69.000000,96000.000000,1.000000


In [ ]:
dataset.head()


,Age,Salary,Purchased
0,18,20000,0
1,19,22000,0
2,20,24000,0
3,21,28000,0
4,22,60000,1


In [ ]:
dataset.iloc[:, :-1]


,Age,Salary
0,18,20000
1,19,22000
2,20,24000
3,21,28000
4,22,60000
...,...,...
1549,66,26000
1550,67,66000
1551,68,80000
1552,69,90000


In [ ]:
X = dataset.iloc[:, :-1].values
X

array([[   18, 20000],
       [   19, 22000],
       [   20, 24000],
       ...,
       [   68, 80000],
       [   69, 90000],
       [   60, 96000]])

In [ ]:
X = dataset.iloc[:, :-1].values # tmaaam ki tmaam rows, then aakhri column k ilava tmam columns. sirf values lo, indices nahi.

y = dataset.iloc[:,-1].values # just last column ki values


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2, random_state=0)


In [ ]:
type(X_train)

numpy.ndarray

In [ ]:
X_train

array([[   43, 62000],
       [   66, 46000],
       [   26, 66000],
       ...,
       [   67, 66000],
       [   28, 62000],
       [   22, 27000]])

In [ ]:
X_train.shape

(1243, 2)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_train = sc.fit_transform(X_train) # i guess
X_test  = sc.transform(X_test)


In [ ]:
# The code snippet you provided is a common preprocessing step in machine learning, where data is standardized before training a model. Here's a detailed breakdown of what each line does:

# 1. **Importing the StandardScaler:**
#    ```python
#    from sklearn.preprocessing import StandardScaler
#    ```
#    This line imports the `StandardScaler` class from the `sklearn.preprocessing` module. The `StandardScaler` is used to standardize features by removing the mean and scaling to unit variance.

# 2. **Creating an instance of StandardScaler:**
#    ```python
#    sc = StandardScaler()
#    ```
#    Here, an instance of the `StandardScaler` is created and assigned to the variable `sc`. This scaler will be used to fit and transform the data.

# 3. **Fitting and transforming the training data:**
#    ```python
#    X_train = sc.fit_transform(X_train)
#    ```
#    This line does two things:
#    - **Fit:** `sc.fit(X_train)` computes the mean and standard deviation of `X_train`.
#    - **Transform:** `sc.transform(X_train)` standardizes `X_train` by subtracting the mean and dividing by the standard deviation for each feature.

#    The `fit_transform` method is a convenient combination of `fit` and `transform`, performing both steps in one go. The transformed training data replaces the original `X_train`.

# 4. **Transforming the test data:**
#    ```python
#    X_test  = sc.transform(X_test)
#    ```
#    This line uses the mean and standard deviation computed from the training data (`X_train`) to transform the test data (`X_test`). It's crucial to use the same parameters (mean and standard deviation) for both training and test data to ensure consistency.

# ### Why Standardize the Data?

# Standardizing the data is a common preprocessing step for many machine learning algorithms because:
# - **Improves Convergence Speed:** Gradient-based algorithms, such as gradient descent, converge faster on standardized data.
# - **Equal Weighting:** Ensures that all features contribute equally to the result, preventing features with larger scales from dominating the model.
# - **Enhances Model Performance:** Some algorithms assume that the data is centered around zero and has a unit variance, and they perform better with standardized data (e.g., support vector machines, k-nearest neighbors, and neural networks).



In [ ]:
X_train[:,0].mean()


5.716353465487532e-17

In [ ]:
X_train

array([[-0.09347628,  0.22152545],
       [ 1.22167788, -0.53416893],
       [-1.06554674,  0.41044905],
       ...,
       [ 1.2788585 ,  0.41044905],
       [-0.95118551,  0.22152545],
       [-1.2942692 , -1.431556  ]])

In [ ]:
X_test[:,0].mean()


-0.09678576465624485

In [ ]:
Xtrain_ = torch.from_numpy(X_train).float()
Xtest_  = torch.from_numpy(X_test).float()

In [ ]:
Xtrain_


tensor([[-0.0935,  0.2215],
        [ 1.2217, -0.5342],
        [-1.0655,  0.4104],
        ...,
        [ 1.2789,  0.4104],
        [-0.9512,  0.2215],
        [-1.2943, -1.4316]])

In [ ]:
ytrain_ = torch.from_numpy(y_train)
ytest_  = torch.from_numpy(y_test)


In [ ]:
ytrain_

tensor([1, 1, 1,  ..., 1, 1, 0])

In [ ]:
ytrain_.shape

torch.Size([1243])

In [ ]:
ytrain_.dim()

1

In [ ]:
Xtrain_.shape, ytrain_.shape

(torch.Size([1243, 2]), torch.Size([1243]))

In [ ]:
Xtest_.shape, ytest_.shape

(torch.Size([311, 2]), torch.Size([311]))

In [ ]:
input_size  = 2    # age and salary
output_size = 2    # 0,1 whether the customer is buying or not?
hidden_size = 10   # 10 neurons in each hidden layer


In [ ]:
# fc: fully conected

class Net(nn.Module):
   def __init__(self):
       super(Net, self).__init__()
       self.fc1 = torch.nn.Linear(input_size, hidden_size)
       self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
       self.fc3 = torch.nn.Linear(hidden_size, output_size)


   def forward(self, X):
       X = self.fc1(X)
       X = torch.relu(X)

       X = self.fc2(X)
       X = torch.relu(X)

       X = self.fc3(X)
       return F.log_softmax(X, dim=1)



In [ ]:
model = Net()


In [ ]:
import torch.optim as optim

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn   = nn.NLLLoss() # negative log likelihood loss


In [ ]:
epochs = 100

for epoch in range(epochs):
  optimizer.zero_grad()        # for each epoch we zero the gradients??

  Ypred = model(Xtrain_)       # making predictions

  loss  = loss_fn(Ypred,  ytrain_)  # computing loss b/w preds and true labels

  loss.backward()                   # perform back propagation?

  optimizer.step()                  # updating the model parameters?

  print('Epoch',epoch, 'loss',loss.item())



Epoch 0 loss 0.6417160630226135
Epoch 1 loss 0.6198132038116455
Epoch 2 loss 0.5994070172309875
Epoch 3 loss 0.5802884101867676
Epoch 4 loss 0.5616825222969055
Epoch 5 loss 0.5434765815734863
Epoch 6 loss 0.5259281992912292
Epoch 7 loss 0.5094735622406006
Epoch 8 loss 0.4944007694721222
Epoch 9 loss 0.4803772568702698
Epoch 10 loss 0.467196524143219
Epoch 11 loss 0.4543963670730591
Epoch 12 loss 0.44128426909446716
Epoch 13 loss 0.42745867371559143
Epoch 14 loss 0.4131253659725189
Epoch 15 loss 0.39863190054893494
Epoch 16 loss 0.38436630368232727
Epoch 17 loss 0.37003839015960693
Epoch 18 loss 0.35587677359580994
Epoch 19 loss 0.3421550989151001
Epoch 20 loss 0.32884302735328674
Epoch 21 loss 0.31595078110694885
Epoch 22 loss 0.30339258909225464
Epoch 23 loss 0.2912221848964691
Epoch 24 loss 0.2792917788028717
Epoch 25 loss 0.2675914466381073
Epoch 26 loss 0.2562026381492615
Epoch 27 loss 0.24520766735076904
Epoch 28 loss 0.2348089963197708
Epoch 29 loss 0.22513382136821747
Epoch 30 l

In [ ]:
model.parameters()


<generator object Module.parameters at 0x7e991d434270>

In [ ]:
list(model.parameters())


[Parameter containing:
 tensor([[-0.3202, -1.1367],
         [ 0.3134,  0.2634],
         [ 0.2629, -0.4789],
         [ 0.0781,  1.2295],
         [ 1.2807,  0.2486],
         [-0.5165, -1.1084],
         [ 0.0060, -0.0555],
         [-0.4958, -0.5429],
         [ 1.1965, -0.2506],
         [-0.3053, -0.2932]], requires_grad=True),
 Parameter containing:
 tensor([ 0.2008,  0.5617,  0.8682,  0.9220,  0.3327,  0.1238, -0.2029,  0.7927,
          0.0414,  0.6984], requires_grad=True),
 Parameter containing:
 tensor([[ 0.5158, -0.1899,  0.1639, -0.5733, -1.0672,  0.5922, -0.0826,  0.3650,
          -0.6781,  0.2393],
         [-0.1378, -0.2800,  0.1345, -0.1762, -0.0192,  0.0210,  0.1534,  0.0748,
          -0.0774, -0.0334],
         [ 0.3396,  0.2439,  0.1026, -0.1708, -0.3734,  0.3170, -0.0458,  0.4877,
          -0.6105,  0.0167],
         [-0.1296,  0.3690,  0.3704,  0.7616,  0.8148, -0.0041, -0.0108,  0.4546,
           0.8923,  0.4167],
         [-0.2316,  0.0237, -0.1350, -0.1249,

# Making predictions

In [ ]:
torch.from_numpy(sc.transform(np.array([[40,20000]]))).float()  # age=20, salary= 20k


tensor([[-0.2650, -1.7622]])

In [ ]:
y_cust_20_40000 = model(torch.from_numpy(sc.transform(np.array([[40,20000]]))).float())
y_cust_20_40000

# [0, 1]
# in our case higher probability associated with zero i.e less -ive


tensor([[-1.6375e-03, -6.4154e+00]], grad_fn=<LogSoftmaxBackward0>)

In [ ]:
_, predicted_20_40000 = torch.max(y_cust_20_40000.data,-1)
predicted_20_40000
# not gonna buy


tensor([0])

In [ ]:
y_cust_42_50000 = model(torch.from_numpy(sc.transform(np.array([[42,50000]]))).float())
y_cust_42_50000


tensor([[-0.8176, -0.5825]], grad_fn=<LogSoftmaxBackward0>)

In [ ]:
_, predicted_42_50000 = torch.max(y_cust_42_50000.data,-1)
predicted_42_50000


tensor([1])

### Saving the model

In [ ]:
#torch.save(model,'customer_buy.pt') # pt: pytorch


In [ ]:
# restored_model = torch.load('customer_buy.pt')

In [ ]:
# y_cust_20_40000 = restored_model(torch.from_numpy(sc.transform(np.array([[40,20000]]))).float())
# y_cust_20_40000

In [ ]:
# _, predicted_20_40000 = torch.max(y_cust_20_40000.data,-1)
# predicted_20_40000

### Can also save just the weights

In [ ]:
model.state_dict()


OrderedDict([('fc1.weight',
              tensor([[ 0.8351,  0.4225],
                      [-0.0681,  1.0825],
                      [-0.6282, -0.3714],
                      [ 0.5764,  0.5442],
                      [-0.7677,  0.7948],
                      [ 0.9796,  0.1000],
                      [-0.8526, -0.5954],
                      [-0.1599,  0.0036],
                      [ 0.1556,  0.8249],
                      [ 0.0069, -1.1019]])),
             ('fc1.bias',
              tensor([ 0.4702,  0.6132,  1.0335,  0.3721,  0.3244,  0.3459,  0.1600, -0.3715,
                       0.1002, -0.2381])),
             ('fc2.weight',
              tensor([[ 0.1620,  0.2613, -0.4059,  0.4108,  0.8249,  0.5653, -0.2541,  0.0541,
                        0.3114,  0.2803],
                      [ 0.0142, -0.1871,  0.0276, -0.1934, -0.1752,  0.1255, -0.3239,  0.0875,
                        0.0983,  0.0590],
                      [-0.9871, -0.7319,  0.8359, -1.2554, -0.5792, -0.9758,  0.2652

In [ ]:
torch.save(model.state_dict(),'customer_buy_state_dict')


### Loading weights into a new model

In [ ]:
new_predictor = Net()


In [ ]:
y_cust_20_40000 = new_predictor(torch.from_numpy(sc.transform(np.array([[40,20000]]))).float())
y_cust_20_40000


tensor([[-0.6273, -0.7636]], grad_fn=<LogSoftmaxBackward0>)

In [ ]:
new_predictor.load_state_dict(torch.load('customer_buy_state_dict'))

<All keys matched successfully>

In [ ]:
y_cust_20_40000 = new_predictor(torch.from_numpy(sc.transform(np.array([[40,20000]]))).float())
y_cust_20_40000


tensor([[-3.1996e-03, -5.7463e+00]], grad_fn=<LogSoftmaxBackward0>)

In [ ]:
!zip -r customer_buy_state_dict.zip customer_buy_state_dict


  adding: customer_buy_state_dict (deflated 52%)


In [ ]:
from google.colab import files


In [ ]:
# files.download('customer_buy_state_dict.zip')
# # perfect

# pytorch call backs

https://chatgpt.com/share/1a49c819-c3a2-403d-9cf6-9fd6418d8afe